# Caderno Jupyter: Estimativa de Profundidade com o Modelo VGGT

Este notebook demonstra como usar o modelo VGGT (Vision-Guided Geometry Transformer) para estimar a profundidade de um conjunto de imagens. O processo é dividido em etapas, desde o carregamento dos dados e do modelo até a inferência e o salvamento dos resultados. É um ótimo ponto de partida para entender o fluxo de trabalho prático ao usar um modelo de deep learning pré-treinado.

## Célula 2: Importação das Bibliotecas

Primeiro, vamos importar todas as bibliotecas necessárias.
* **`argparse`**: Usado no script original para ler argumentos da linha de comando. No notebook, vamos simular isso para facilitar a alteração dos parâmetros.
* **`logging`**: Para exibir mensagens informativas sobre o progresso da execução.
* **`pathlib`**: Para manipular caminhos de arquivos e diretórios de forma mais intuitiva.
* **`time`**: Para medir o tempo de execução das principais partes do código.
* **`numpy`**: Uma biblioteca fundamental para computação numérica em Python, usada aqui para manipular os mapas de profundidade.
* **`PIL (Pillow)`**: Usada para salvar as imagens de profundidade em formato PNG.
* **`matplotlib.pyplot`**: Para criar e salvar visualizações coloridas dos mapas de profundidade.
* **`torch`**: A biblioteca de deep learning que usaremos para carregar e executar o modelo VGGT.
* **`vggt`**: A biblioteca específica do modelo que contém a arquitetura do VGGT e funções úteis para pré-processamento de imagens.

In [1]:
# Célula 3: Importação das Bibliotecas
import argparse
import logging
from pathlib import Path
import time

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import torch

from vggt.models.vggt import VGGT
from vggt.utils.load_fn import load_and_preprocess_images

/home/segreto/miniconda3/envs/vggt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/segreto/miniconda3/envs/vggt/lib/python3.11/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


## Célula 4: Funções Utilitárias

Nesta seção, definimos algumas funções auxiliares que nos ajudarão ao longo do processo.

* **`setup_logger`**: Configura como as mensagens de log (avisos, informações, etc.) serão exibidas.
* **`collect_images`**: Procura recursivamente por todos os arquivos de imagem (com extensões como `.png`, `.jpg`) dentro de um diretório e seus subdiretórios.
* **`save_color_with_axes_cm`**: Salva o mapa de profundidade como uma imagem colorida. Ela usa um mapa de cores (como "turbo") para representar a profundidade e adiciona uma barra de cores com legendas em centímetros, o que facilita a interpretação visual.
* **`save_mm_png`**: Salva o mapa de profundidade como uma imagem PNG de 16 bits em tons de cinza. Cada valor de pixel nesta imagem corresponde à profundidade em milímetros. Este formato é ideal para armazenamento preciso dos dados, embora a imagem possa parecer escura em visualizadores comuns.

In [2]:
# Célula 5: Funções Utilitárias

# Define as extensões de arquivo de imagem que vamos procurar
IMG_EXTS = {".png", ".jpg", ".jpeg", ".PNG", ".JPG", ".JPEG"}

def setup_logger(verbosity: int):
    """Configura o nível de detalhe das mensagens de log."""
    level = logging.WARNING if verbosity == 0 else logging.INFO if verbosity == 1 else logging.DEBUG
    logging.basicConfig(
        level=level,
        format="%(asctime)s | %(levelname)-8s | %(message)s",
        datefmt="%H:%M:%S",
    )

def collect_images(root: Path) -> list[Path]:
    """Coleta recursivamente os arquivos de imagem de um diretório."""
    if not root.exists():
        raise FileNotFoundError(f"Pasta de entrada não encontrada: {root}")
    files = [p for p in sorted(root.rglob("*")) if p.suffix in IMG_EXTS and p.is_file()]
    return files

def save_color_with_axes_cm(depth_m: np.ndarray, out_path: Path, ticks: int = 9, cmap_name: str = "turbo"):
    """
    Salva uma imagem de profundidade colorida com eixos e uma barra de cores em centímetros.
    """
    d_m = depth_m.copy()
    # Define um intervalo robusto para a visualização, ignorando valores extremos (outliers)
    p1, p99 = np.nanpercentile(d_m, 1), np.nanpercentile(d_m, 99)
    if not np.isfinite(p1) or not np.isfinite(p99) or p99 <= p1:
        p1, p99 = float(np.nanmin(d_m)), float(np.nanmax(d_m))
    if not np.isfinite(p1) or not np.isfinite(p99) or p99 <= p1:
        p1, p99 = 0.0, 1.0

    fig, ax = plt.subplots(figsize=(8, 5), dpi=150)
    im = ax.imshow(d_m, cmap=cmap_name, vmin=p1, vmax=p99, interpolation="nearest")
    ax.set_title("Profundidade Prevista")
    ax.set_xlabel("u (pixels)")
    ax.set_ylabel("v (pixels)")
    cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

    # Cria marcações na barra de cores em centímetros
    ticks_m = np.linspace(p1, p99, num=max(3, ticks))
    ticks_cm = ticks_m * 100.0
    cbar.set_ticks(ticks_m)
    cbar.set_ticklabels([f"{t:.0f}" for t in ticks_cm])
    cbar.set_label("Profundidade (cm)", rotation=90)

    fig.tight_layout()
    fig.savefig(out_path.as_posix())
    plt.close(fig)

def save_mm_png(depth_m: np.ndarray, out_path: Path, scale_m: float):
    """Salva a profundidade como uma imagem PNG de 16 bits em milímetros."""
    mm = np.clip(depth_m * scale_m * 1000.0, 0, 65535).astype(np.uint16)
    Image.fromarray(mm, mode="I;16").save(out_path.as_posix())

## Célula 6: Configuração dos Parâmetros

Em um script Python, os parâmetros são geralmente passados pela linha de comando. Como estamos em um notebook, definimos esses parâmetros diretamente em uma classe `Args`. Isso facilita a experimentação: você pode simplesmente alterar os valores nesta célula e executar o restante do notebook novamente.

**Parâmetros importantes:**
* `input`: O caminho para a pasta que contém as imagens que você deseja processar.
* `output`: O caminho para a pasta onde os resultados serão salvos.
* `model_id`: O identificador do modelo pré-treinado a ser usado. O padrão é `"facebook/VGGT-1B"`, que será baixado do Hugging Face Hub.
* `device`: O dispositivo para executar a inferência. Mude para `"cpu"` se você não tiver uma GPU NVIDIA compatível.
* `batch_size`: O número de imagens a serem processadas de uma só vez. Um valor maior pode ser mais rápido, mas consome mais memória da GPU. Se você encontrar erros de "Out of Memory" (OOM), reduza este valor.

In [9]:
# Célula 7: Configuração dos Parâmetros

class Args:
    # !!! MODIFIQUE OS CAMINHOS DE ENTRADA E SAÍDA ABAIXO !!!
    input = Path("./images")
    output = Path("./output")
    
    # --- Outros parâmetros ---
    # Identificador do modelo no Hugging Face Hub ou caminho local
    model_id = "facebook/VGGT-1B"
    # Dispositivo de inferência ('cuda' para GPU NVIDIA, 'cpu' para CPU)
    device = "cuda"
    # Fator de escala métrica para a profundidade (mantenha 1.0 se desconhecido)
    scale_m = 1.0
    # Salvar também em formato PNG de 16 bits (milímetros)
    save_mm_png = False
    # Limitar o número total de imagens a processar (útil para testes rápidos)
    max_views = None # Ex: 10 para processar apenas as 10 primeiras imagens
    # Tamanho do lote (batch size) para processamento
    batch_size = 16
    # Mapa de cores para as visualizações
    colormap = "turbo"
    # Número de marcações na barra de cores
    cb_ticks = 9
    # Nível de verbosidade do log (0: warning, 1: info, 2: debug)
    verbose = 1

args = Args()

# Configura o logger e cria a pasta de saída
setup_logger(args.verbose)
args.output.mkdir(parents=True, exist_ok=True)

## Célula 8: Preparação do Ambiente e Carregamento de Dados

Nesta etapa, preparamos o ambiente para a inferência:

1.  **Seleção do Dispositivo e `dtype`**: O código verifica se uma GPU CUDA está disponível e define o dispositivo (`cuda` ou `cpu`). Ele também seleciona o tipo de dados (`dtype`) para a computação. O uso de `bfloat16` ou `float16` (precisão mista) em GPUs pode acelerar a inferência e reduzir o uso de memória sem uma perda significativa de precisão.
2.  **Coleta das Imagens**: A função `collect_images` é chamada para encontrar todos os arquivos de imagem na pasta de entrada definida anteriormente.
3.  **Carregamento do Modelo**: O modelo VGGT pré-treinado é baixado e carregado na memória. `.to(device)` move o modelo para a GPU (ou CPU), e `.eval()` o coloca em modo de avaliação, o que desativa camadas como o dropout, que são usadas apenas durante o treinamento.

In [10]:
# Célula 9: Preparação do Ambiente e Carregamento de Dados

# 1. Seleção do Dispositivo e Tipo de Dados (dtype) para Precisão Mista (AMP)
device = torch.device(args.device)
if torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 8:
    amp_dtype = torch.bfloat16
else:
    amp_dtype = torch.float16 if device.type == "cuda" else torch.float32
logging.info(f"Dispositivo: {device} | dtype para AMP: {amp_dtype}")

# 2. Coleta das Imagens
logging.info("Coletando imagens...")
image_paths = collect_images(args.input)
if args.max_views is not None:
    image_paths = image_paths[: args.max_views]

if not image_paths:
    logging.error(f"Nenhuma imagem encontrada em {args.input}")
else:
    logging.info(f"Encontradas {len(image_paths)} imagens.")
    # Mostra as 5 primeiras imagens como exemplo
    for p in image_paths[:5]:
        logging.debug(f"  {p}")
    if len(image_paths) > 5:
        logging.debug("  ...")

    # 3. Carregamento do Modelo
    logging.info(f"Carregando o modelo: {args.model_id}")
    model = VGGT.from_pretrained(args.model_id).to(device).eval()

14:51:55 | INFO     | Dispositivo: cuda | dtype para AMP: torch.bfloat16
14:51:55 | INFO     | Coletando imagens...
14:51:55 | INFO     | Encontradas 138 imagens.
14:51:55 | INFO     | Carregando o modelo: facebook/VGGT-1B
14:51:55 | INFO     | using MLP layer as FFN


## Célula 10: Pré-processamento das Imagens

Antes de passar as imagens pelo modelo, elas precisam ser pré-processadas. Isso geralmente envolve:
* Redimensionar as imagens para o tamanho esperado pelo modelo.
* Normalizar os valores dos pixels (por exemplo, para o intervalo [0, 1] e depois subtrair a média e dividir pelo desvio padrão).
* Converter as imagens em tensores do PyTorch.

A função `load_and_preprocess_images` cuida de todo esse processo para nós, criando um único tensor gigante com todas as imagens prontas para a inferência.

In [12]:
# Célula 11: Pré-processamento das Imagens

if image_paths:
    # Converte os caminhos para strings
    image_strs = [p.as_posix() for p in image_paths]
    
    logging.info("Pré-processando as imagens...")
    # Carrega e pré-processa todas as imagens, movendo-as para o dispositivo selecionado
    images = load_and_preprocess_images(image_strs).to(device)  # Formato: (N, 3, H, W)
    logging.info(f"Tensor de imagens criado com o formato: {tuple(images.shape)}")

14:52:27 | INFO     | Pré-processando as imagens...
14:52:29 | INFO     | Tensor de imagens criado com o formato: (138, 3, 392, 518)


## Célula 12: Execução da Inferência em Lotes (Batch Processing)

Esta é a célula principal onde a "mágica" acontece. Para evitar sobrecarregar a memória da GPU, processamos as imagens em lotes (`chunks`).

O loop `for` itera sobre o tensor de imagens, pegando um `batch_size` de cada vez. Para cada lote:
1.  **`torch.no_grad()`**: Desativa o cálculo de gradientes, o que economiza memória e acelera a computação, já que não estamos treinando o modelo.
2.  **`torch.cuda.amp.autocast()`**: Habilita a precisão mista automática, que realiza operações em `float16` ou `bfloat16` sempre que possível para acelerar o processo.
3.  **`model.aggregator`**: A primeira parte do modelo VGGT, que extrai e agrega características de múltiplas visualizações (imagens).
4.  **`model.depth_head`**: A segunda parte, que usa as características agregadas para prever o mapa de profundidade final.
5.  **Salvamento dos Resultados**: Para cada imagem no lote processado, o mapa de profundidade resultante é:
    * Convertido para um array NumPy.
    * Salvo como um arquivo `.npy` (dados brutos), uma imagem colorida `_color.png` e, opcionalmente, um PNG de 16 bits `_mm.png`.

In [13]:
# Célula 13: Execução da Inferência em Lotes (Batch Processing)

if image_paths:
    N_total = images.shape[0]
    bs = max(1, args.batch_size)
    logging.info(f"Processando em lotes (chunks) de {bs} imagens para evitar falta de memória (OOM).")

    save_idx = 0
    t_all0 = time.perf_counter()

    # Itera sobre as imagens em lotes
    for start in range(0, N_total, bs):
        end = min(start + bs, N_total)
        chunk = images[start:end]
        n = chunk.shape[0]
        logging.info(f"Processando lote {start:04d}:{end:04d} ({n} imagens)")

        # Contextos de otimização para inferência
        with torch.no_grad():
            # Adiciona uma dimensão de lote (B=1) para o modelo
            images_batched = chunk.unsqueeze(0)  # (B=1, n, 3, H, W)
            logging.debug(f"Formato do lote de entrada: {tuple(images_batched.shape)}")

            t0 = time.perf_counter()
            with torch.cuda.amp.autocast(dtype=amp_dtype):
                # 1. Passa as imagens pelo agregador do modelo
                aggregated_tokens_list, ps_idx = model.aggregator(images_batched)
            t1 = time.perf_counter()
            logging.info(f"  Tempo do Aggregator: {t1 - t0:.3f}s")

            t2 = time.perf_counter()
            with torch.cuda.amp.autocast(dtype=amp_dtype):
                # 2. Passa os tokens agregados para a cabeça de profundidade
                depth_map, depth_conf = model.depth_head(aggregated_tokens_list, images_batched, ps_idx)
            t3 = time.perf_counter()
            logging.info(f"  Tempo do Depth Head: {t3 - t2:.3f}s | Total do Lote: {t3 - t0:.3f}s")

        # Garante que o formato do mapa de profundidade está correto
        if depth_map.dim() == 5 and depth_map.size(-1) == 1:
            depth_map = depth_map.squeeze(-1)
        if depth_map.dim() != 4 or depth_map.shape[0] != 1 or depth_map.shape[1] != n:
            raise RuntimeError(f"Formato inesperado do mapa de profundidade: {tuple(depth_map.shape)}")

        # Salva os resultados para cada imagem deste lote
        for i in range(n):
            d_raw = depth_map[0, i].float().cpu().numpy()  # Profundidade em metros (escala relativa)
            d_metric = d_raw * args.scale_m               # Profundidade em metros (escala correta)

            d_min = float(np.nanmin(d_metric))
            d_mean = float(np.nanmean(d_metric))
            d_max = float(np.nanmax(d_metric))
            logging.info(f"  Imagem {save_idx:03d} | prof. min/média/max (m): {d_min:.4f}/{d_mean:.4f}/{d_max:.4f}")

            base = f"depth_{save_idx:03d}"
            # Salva a profundidade métrica como .npy
            np.save(args.output / f"{base}.npy", d_metric.astype(np.float32))
            # Salva a visualização colorida com barra de cores em cm
            save_color_with_axes_cm(d_metric, args.output / f"{base}_color.png", ticks=args.cb_ticks, cmap_name=args.colormap)
            # Salva opcionalmente o PNG de 16 bits em milímetros
            if args.save_mm_png:
                save_mm_png(d_raw, args.output / f"{base}_mm.png", scale_m=args.scale_m)

            save_idx += 1

    t_all1 = time.perf_counter()
    logging.info(f"Processamento concluído. {save_idx} imagens processadas em {t_all1 - t_all0:.3f}s")
    logging.info(f"Resultados salvos em: {args.output.as_posix()}")

14:52:31 | INFO     | Processando em lotes (chunks) de 16 imagens para evitar falta de memória (OOM).
14:52:31 | INFO     | Processando lote 0000:0016 (16 imagens)
14:52:32 | INFO     |   Tempo do Aggregator: 1.003s
14:52:32 | INFO     |   Tempo do Depth Head: 0.161s | Total do Lote: 1.165s
14:52:32 | INFO     |   Imagem 000 | prof. min/média/max (m): 0.3402/0.8932/1.4779
14:52:32 | INFO     |   Imagem 001 | prof. min/média/max (m): 0.2865/0.8903/1.4664
14:52:32 | INFO     |   Imagem 002 | prof. min/média/max (m): 0.3272/0.8896/1.4607
14:52:32 | INFO     |   Imagem 003 | prof. min/média/max (m): 0.1821/0.8835/1.4693
14:52:32 | INFO     |   Imagem 004 | prof. min/média/max (m): 0.2671/0.8873/1.4607
14:52:33 | INFO     |   Imagem 005 | prof. min/média/max (m): 0.2302/0.8806/1.4635
14:52:33 | INFO     |   Imagem 006 | prof. min/média/max (m): 0.2214/0.8818/1.4635
14:52:33 | INFO     |   Imagem 007 | prof. min/média/max (m): 0.3350/0.8827/1.4635
14:52:33 | INFO     |   Imagem 008 | prof. m